# Central Bank Cuts and Hikes
© 2026 Jim Domeij 

This query returns the monthly count of interest rate hikes and cuts across G20 countries since end-2016, illustrating shifts in global monetary policy between tightening and easing during this period

In [5]:
from polars_bloomberg import BQuery

query = """
let(
    #ref_date = policy_rate().period_reference_date;
    #policy_rate_changes = diff(dropNA(policy_rate(), remove_id=True));
    #count_cuts = sum(group(if(#policy_rate_changes < 0, 1, 0), by=#ref_date));
    #count_hikes = sum(group(if(#policy_rate_changes > 0, 1, 0), by=#ref_date));
    #net_hike_count = #count_hikes - #count_cuts;
)
get(#count_cuts, #count_hikes, #net_hike_count)
for(countries('G20'))
with(pr=range(2016M12, 2025M12), pt=M, act_est_data=A, fill=PREV)
preferences(dropCols=["ORIG_IDS"])
"""

with BQuery() as bq:
    res = bq.bql(query)
df = res.combine()
df.head()

ID,#count_cuts,#REF_DATE,#count_hikes,#net_hike_count
str,f64,date,f64,f64
"""2016-12-31T00-00-00Z""",0.0,2016-12-31,0.0,0.0
"""2017-01-31T00-00-00Z""",1.0,2017-01-31,0.0,-1.0
"""2017-02-28T00-00-00Z""",1.0,2017-02-28,1.0,0.0
"""2017-03-31T00-00-00Z""",1.0,2017-03-31,2.0,1.0
"""2017-04-30T00-00-00Z""",2.0,2017-04-30,0.0,-2.0


In [19]:
# create chart
import altair as alt
import polars as pl

g_data = df.select(
    pl.col("#REF_DATE").alias("date"),
    pl.col("#count_cuts").mul(-1).alias("cuts"),
    pl.col("#count_hikes").alias("hikes"),
    pl.col("#net_hike_count").alias("net"),
)

base = alt.Chart(g_data).encode(x=alt.X("date:T").title(None))

bars = (
    base.transform_fold(["cuts", "hikes"], as_=["type", "count"])
    .mark_bar()
    .encode(
        y=alt.Y("count:Q").title("Number of Rate Changes").title("Count"),
        color=alt.Color(
            "type:N",
            title="Type of Rate Change",
            scale=alt.Scale(domain=["hikes", "cuts"], range=["#1f77b4", "#ff7f0e"]),
        ),
    )
)
line = base.mark_line(color="black", size=1).encode(y=alt.Y("net:Q", title="Count"))

bars + line

alt.LayerChart(...)